In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

conn = sqlite3.connect('/home/kub/projects/ML/Bezrealitky.db')
df = pd.read_sql_query("SELECT * FROM tab_parametry", conn)
df

,Cislo_inzeratu,Dispozice,Stav,Plocha,Cena,Mesto,Mestska,Vlastnictvi,Typ,PENB,...,Sklep,Lodzie,Parkovani,Vytah,Garaz,K,Rekonstrukce,Odkaz,Datum_pridani,Datum_odebrani
0,688885,3+kk,Špatný,60 m²,6.750.000 Kč,Praha,Kobylisy,Osobní,Cihla,G,...,Ano,Ne,Ne,Ne,Ne,15. 10. 2021,-,/home/kub/projects/bezrealitky/fotky/1,18:03:10,-
1,688849,2+kk,Velmi dobrý,62 m²,7.390.000 Kč,Praha,Běchovice,Osobní,Cihla,G,...,Ano,Ne,Ne,Ano,Ne,01. 02. 2022,-,/home/kub/projects/bezrealitky/fotky/2,18:03:10,-
2,688255,1+1,Dobrý,47 m²,5.100.000 Kč,Praha,Žižkov,Osobní,Cihla,D,...,Ano,Ne,Ano,Ano,Ne,30. 10. 2021,exteriér,/home/kub/projects/bezrealitky/fotky/3,18:03:10,-
3,688244,3+1,Novostavba,74 m²,1.990.000 Kč,Karviná,Karviná-město,Družstevní,Panel,G,...,Ano,Ne,Ne,Ano,Ne,14. 10. 2021,-,/home/kub/projects/bezrealitky/fotky/4,18:03:10,-
4,687224,3+1,Velmi dobrý,74 m²,7.890.000 Kč,Praha,Chodov,Osobní,Panel,C,...,Ano,Ano,Ano,Ano,Ne,01. 12. 2021,-,/home/kub/projects/bezrealitky/fotky/5,18:03:10,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,670842,3+kk,Dobrý,127 m²,8.925.000 Kč,Slaný,-,Osobní,Cihla,-,...,Ano,Ano,Ne,Ne,Ano,-,-,/home/kub/projects/bezrealitky/fotky/185,18:03:10,-
185,688412,3+kk,Novostavba,74 m²,9.490.000 Kč,Praha,Hodkovičky,Osobní,Ostatní,B,...,Ne,Ne,Ano,Ano,Ano,15. 10. 2021,-,/home/kub/projects/bezrealitky/fotky/186,18:03:10,-
186,686982,1+kk,Novostavba,31 m²,3.999.000 Kč,Praha,Zličín,Osobní,Cihla,G,...,Ano,Ne,Ano,Ano,Ano,09. 10. 2021,-,/home/kub/projects/bezrealitky/fotky/187,18:03:10,-
187,662285,3+kk,Novostavba,82 m²,14.950.000 Kč,Praha,Košíře,Osobní,Cihla,-,...,Ano,Ano,Ne,Ano,Ano,-,-,/home/kub/projects/bezrealitky/fotky/188,18:03:10,-


In [2]:
"""
V modelu bude:
cena = 
dispozice



stav

plocha - v m2

Praha (ano/ne) - 1/0

vlastnictvi - 1 osobní, 0 družstevní


vybavenost

PENB - G-1, F-2, E-3, D-4, C-5, B-6, A-7

balkon - 1/0
terasa- 1/0
sklep - 1/0
lodzie - 1/0
parkovani - 1/0
terasa - 1/0
vytah - 1/0
garaz - 1/0
"""


'\nV modelu bude:\ncena = \ndispozice\n\n\n\nstav\n\nplocha - v m2\n\nPraha (ano/ne) - 1/0\n\nvlastnictvi - 1 osobní, 0 družstevní\n\n\nvybavenost\n\nPENB - G-1, F-2, E-3, D-4, C-5, B-6, A-7\n\nbalkon - 1/0\nterasa- 1/0\nsklep - 1/0\nlodzie - 1/0\nparkovani - 1/0\nterasa - 1/0\nvytah - 1/0\ngaraz - 1/0\n'

In [20]:
df2 = df
df2 = df2.drop(columns = ["Cislo_inzeratu", "Mestska", "Podlazi", "Typ", "K", "Rekonstrukce", "Odkaz", "Datum_pridani", "Datum_odebrani"])
df2

df2.columns = df2.columns.str.replace("Vlastnictvi", "Osobni_vl")
df2.loc[(df2.Osobni_vl == "Osobní"), "Osobni_vl"] = 1 #1 pro osobní vlastnictví, 0 pro družstevní
df2.loc[(df2.Osobni_vl != 1),        "Osobni_vl"] = 0

df2.columns = df2.columns.str.replace("Mesto", "Praha")
df2.loc[(df2.Praha == "Praha"), "Praha"] = 1 #1 pro Prahu, 0 pro zbytekí
df2.loc[(df2.Praha != 1), "Praha"] = 0

df2.loc[(df2.PENB == "G"), "PENB"] = 1 #
df2.loc[(df2.PENB == "F"), "PENB"] = 2
df2.loc[(df2.PENB == "E"), "PENB"] = 3
df2.loc[(df2.PENB == "D"), "PENB"] = 4
df2.loc[(df2.PENB == "C"), "PENB"] = 5
df2.loc[(df2.PENB == "B"), "PENB"] = 6
df2.loc[(df2.PENB == "A"), "PENB"] = 7
df2.loc[(df2.PENB == "-"), "PENB"] = 1

df2.loc[(df2.Vybavenost == "Nevybavený"), "Vybavenost"] = 1
df2.loc[(df2.Vybavenost == "Částečně"), "Vybavenost"] = 2
df2.loc[(df2.Vybavenost == "Vybavený"), "Vybavenost"] = 3
df2.loc[(df2.Vybavenost == "-"), "Vybavenost"] = 1

anone_parametry = ["Balkon", "Terasa", "Sklep", "Lodzie", "Parkovani", "Vytah", "Garaz"]

df2.loc[(df2.Balkon == "Ano"), "Balkon"] = 1
df2.loc[(df2.Balkon == "Ne"),  "Balkon"] = 0

df2.loc[(df2.Terasa == "Ano"), "Terasa"] = 1
df2.loc[(df2.Terasa == "Ne"),  "Terasa"] = 0

df2.loc[(df2.Sklep == "Ano"), "Sklep"] = 1
df2.loc[(df2.Sklep == "Ne"),  "Sklep"] = 0

df2.loc[(df2.Lodzie == "Ano"), "Lodzie"] = 1
df2.loc[(df2.Lodzie == "Ne"),  "Lodzie"] = 0

df2.loc[(df2.Parkovani == "Ano"), "Parkovani"] = 1
df2.loc[(df2.Parkovani == "Ne"),  "Parkovani"] = 0

df2.loc[(df2.Vytah == "Ano"), "Vytah"] = 1
df2.loc[(df2.Vytah == "Ne"),  "Vytah"] = 0

df2.loc[(df2.Garaz == "Ano"), "Garaz"] = 1
df2.loc[(df2.Garaz == "Ne"),  "Garaz"] = 0

df2["Plocha"] = df2['Plocha'].str[:-2].astype(int) #zbavim se m2
df2['Cena'] = df2['Cena'].str.replace('.','')
df2["Cena"] = df2['Cena'].str[:-2].astype(int)

df2.loc[(df2.Stav == "Špatný"), "Stav"] = 1
df2.loc[(df2.Stav == "Dobrý"), "Stav"] = 2
df2.loc[(df2.Stav == "Velmi dobrý"), "Stav"] = 3
df2.loc[(df2.Stav == "Ve výstavbě"), "Stav"] = 4
df2.loc[(df2.Stav == "Novostavba"), "Stav"] = 5
df2.loc[(df2.Stav == "-"), "Stav"] = 3

df2.loc[(df2.Dispozice == "Ostatní"), "Dispozice"] = 1
df2.loc[(df2.Dispozice == "1+kk"),    "Dispozice"] = 1
df2.loc[(df2.Dispozice == "1+1"),     "Dispozice"] = 1
df2.loc[(df2.Dispozice == "2+1"),     "Dispozice"] = 2
df2.loc[(df2.Dispozice == "2+kk"),    "Dispozice"] = 2
df2.loc[(df2.Dispozice == "3+1"),     "Dispozice"] = 3
df2.loc[(df2.Dispozice == "3+kk"),    "Dispozice"] = 3
df2.loc[(df2.Dispozice == "4+1"),     "Dispozice"] = 4
df2.loc[(df2.Dispozice == "4+kk"),    "Dispozice"] = 4
df2.loc[(df2.Dispozice == "5+1"),     "Dispozice"] = 5
df2.loc[(df2.Dispozice == "5+kk"),    "Dispozice"] = 5

df2

/tmp/ipykernel_34840/1741835562.py:51: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['Cena'] = df2['Cena'].str.replace('.','')


,Dispozice,Stav,Plocha,Cena,Praha,Osobni_vl,PENB,Vybavenost,Balkon,Terasa,Sklep,Lodzie,Parkovani,Vytah,Garaz
0,3,1,60,6750000,1,1,1,1,0,0,1,0,0,0,0
1,2,3,62,7390000,1,1,1,2,0,1,1,0,0,1,0
2,1,2,47,5100000,1,1,4,1,0,0,1,0,1,1,0
3,3,5,74,1990000,0,0,1,1,0,0,1,0,0,1,0
4,3,3,74,7890000,1,1,5,3,1,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,3,2,127,8925000,0,1,1,1,1,0,1,1,0,0,1
185,3,5,74,9490000,1,1,6,2,1,0,0,0,1,1,1
186,1,5,31,3999000,1,1,1,3,0,0,1,0,1,1,1
187,3,5,82,14950000,1,1,1,1,0,0,1,1,0,1,1


In [27]:
X = df2[["Dispozice", "Stav", "Plocha", "Praha", "Osobni_vl", "PENB", 
        "Vybavenost", "Balkon", "Terasa", "Sklep", "Lodzie", "Parkovani", "Garaz"]]
y = df2[["Cena"]]

regr = LinearRegression()
regr.fit(X, y)

print("Intercept: ", regr.intercept_)
print("Coefficients: ", regr.coef_)

#y_pred = regressor.predict(X)

Intercept:  [-3377466.96239172]
Coefficients:  [[  88149.70985721  341812.57083239   74677.45098719 3572811.99797143
   865427.97577407  117382.93140072   65022.77587407  194122.2297919
   553444.25770574  160024.64931493  -38045.90363238 -675267.894756
  1418155.45871722]]


In [28]:
test_byt = [3, 3, 79, 1, 1, 1, 3, 0, 0, 1, 1, 0, 0]
regr.predict([test_byt])


/home/kub/.local/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[8684608.4861164]])

Dispozice  Stav  Plocha  Cena     Praha  Osobni_vl  PENB  Vybavenost  Balkon  Terasa  Sklep  Lodzie  Parkovani  Vytah  Garaz
1          1     35      2800000  0      1          -     -           0       0       0      0       0          1      0        1
3          3     70      6081000  1      0          -     -           0       0       1      1       0          1      0        1
                 62      7850000  1      1          1     3           0       0       1      1       0          1      0        1
                 63      3300000  0      1          5     1           0       0       1      0       1          0      1        1
                 64      2399000  0      1          1     1           0       0       1      1       0          1      0        1
                                                                                                                               ..
2          3     51      6790000  1      1          1     1           0       0       1      0 